In [6]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [57]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator
video_id = 'zBUhQPPS9AY'



def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''


tran = get_transcript(video_id)


justText = ''
textAndTimestamps = ''
for text in tran:
    textAndTimestamps += str(int(text['start'])) + '\n' + text['text'] + '\n'
    justText += text['text'] + '\n'



In [58]:
node_prefix = 19
node_start_sym = 'a'
topics = [{
	"start": 27,
	"finish": 109 
},{
	"start": 246,
	"finish": 411
},{
	"start": 411,
	"finish": 610 
},{
	"start": 610,
	"finish": 836 
},{
	"start": 836,
	"finish": 932
},{
	"start": 934,
	"finish": 1024
},{
	"start": 1024,
	"finish": 1084
},{
	"start": 1084,
	"finish": 1230
},{
	"start": 1230,
	"finish": 1318
},{
	"start": 1318,
	"finish": 1423
},{
	"start": 1423,
	"finish": 1495 
},{
	"start": 1509,
	"finish": 1595 
},{
	"start": 1595,
	"finish": 1680
}
]






In [59]:
import os

startNum = node_prefix
startChar = node_start_sym
path = '/Users/shota/github/content-ge/startup-school/nodes'


def make_folder(name):
    newNode = os.path.join(path, name)
    if not os.path.exists(newNode):
        os.makedirs(newNode)
    newFile = os.path.join(newNode, "text1.md")
    f = open(newFile, 'w')
    f.close()
    


for topic in topics:
    name = str(startNum) + startChar
    print('"' + name + '": ' + json.dumps(getTopic(topic, video_id), indent=4) + ',')
    make_folder(name)
    startChar = chr(ord(startChar) + 1)


"19a": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "zBUhQPPS9AY",
            "start": 27,
            "end": 109
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"19b": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "zBUhQPPS9AY",
            "start": 246,
            "end": 411
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"19c": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "zBUhQPPS9AY",
            "start": 411,
            "end": 610
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"19d": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    

In [60]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a topic. Your goal is to produce:
1. Name of the topic
2. Very very small description of the topic
3. Rewrite this topic from your own perspective, user simple words, use markdown to highlight. Be the best 1-on-1 tutor
"""

def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start']
    end = topic['finish']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text

def getTopic(topic, video_id):
    return {
        "name": "",
        "desctiption": "",
        "texts": ["text1.md"],
        "video_parts": [{
            "id": video_id,
            "start": topic["start"],
            "end": topic["finish"]
        }],
        "node_type": "lecture",
        "parent_id": "startup-school",
        "thumbnail": ""
    }


startNum = node_prefix
startChar = node_start_sym

for topic in topics:
    t = getTranscriptForTopic(topic, tran)
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemsText
            },
            {
              "role": "user",
                "content": t
            }
        ],)
    print("====================================")
    name = str(startNum) + startChar
    newFile = os.path.join(os.path.join(path, name), "text1.md")
    f = open(newFile, 'w')
    print(resp.choices[0].message.content)
    f.write(resp.choices[0].message.content)
    f.close()
    startChar = chr(ord(startChar) + 1)
    print("====================================")

1. Topic Name: **Fundraising for Startups**
   
2. Short Description: **This topic discusses various resources and strategies about fundraising for startup ventures. Additionally, it introduces common platforms for raising funds and different types of investors.**

3. Simplified Explanation:

Okay, let's delve into something really interesting and important for startups: **raising money**. Turns out, lots of people ask us about this subject, probably because it's one of the hardest parts of getting a startup off the ground, right next to creating a product or service that people love.

Now, over the years, there's been boatloads of awesome advice available from **YC (Y Combinator)** about fundraising. I'll highlight some of these resources for you to check out later.

First, we've got some truly insightful essays by **Paul Graham**. He's touched on topics like how to survive the fundraising process, how to secure startup funding, and even how to persuade investors to hitch their wagon 

1. Topic Name: Understanding Startup Fundraising
2. Small Description: Exploring the reality of startup funding processes, breaking down the myths, and understanding the different stages of funding. 
3. Rewrite:

Alright folks, let me make a point to clear up a misunderstanding about raising funds for a startup. You often might think it's a complex, slow, and expensive process owing to the big numbers you see in news articles about companies raising massive amounts from renowned venture capitalists.

So, why does this view exist? Just glance at yesterday's tech news. Companies like Shopify, Mark Vision, Locket-Sprague are spoken about, and the common factor they all share are the huge fundraising numbers. But, what if you're a budding entrepreneur thinking, "I just need $25,000 to quit my regular job and concentrate on my startup"? You might start thinking this whole fundraising business isn't for you.

Here's the fun fact, though: What you typically see in big news headlines are Serie

1. Topic Name: **The Pros and Cons of Bootstrapping in Startups**

2. Very Very Small Description: This topic discusses the concept of bootstrapping in startups, the challenges associated with it, and why raising funds can be a more beneficial approach.

3. Rewrite of the Topic:

So, let's talk about **bootstrapping** in your brand-new business. You might be thinking it's the way to go, and sure, everyone does it at the beginning. That's where we all start, without a doubt. But, is it the best route for the long haul? Well, I'm here to tell you, always relying solely on bootstrapping can be tough. 

Now, what exactly is bootstrapping? Good question! It's the process of **building your startup using only the money that you get from your clients**. It can give you a sense of independence, but it's not always a sunny day at the beach.

Here's why it's not all rainbows and sunshine:

- **Universal Fear Factor:** You're always on the edge, scared of running out of funds any day. It's distre

1. Topic Name: Addressing Common Misconceptions about Starting a Startup
2. Description: This entails debunking prevalent myths that discourage potential startup founders including the notions of extensive networks, complex pitches, immediate funding etc. The aim is to encourage self-belief and the drive to simply start and persist.

3. Rewrite:

Okay, let's have a breather and put a bow on our chat. Can you see a thread running through these myths we've chatted about? There's a huge myth, the one that throws a blanket over all these little beliefs. That myth says, "This startup thing, it's not your cup of tea." All these wrong ideas and mixed-up thoughts that founders carry around, they add up. And they may make you think, "Hang on, maybe this startup gig isn't my thing. I don't know how to do this. I don't have a swanky network of influencers and I don't rub shoulders with the well-connected crowd. I'm clueless on how to pitch an idea or dazzle people with my brilliance." 

But liste